In [ ]:
import os
import sys
import cryptography.hazmat
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import CTR # Might use, has parallel encryption
from cryptography.hazmat.primitives.ciphers.modes import CBC
from cryptography.hazmat.primitives.ciphers.modes import XTS # Only for AES, but this is disk encryption mode
from cryptography.hazmat.backends import default_backend

In [ ]:
# Fernet needs a key in constructor
# So lets build one from a key derivation scheme
# I'm using PBKDF2

# A few notes
# AES with 128 bit keys is fine
# So we only need 128/8 = 16 bytes of key
# We, however, will use AES256 because we can
# In the worst case, 256 performs like it has 131 bits of key

# Because we are using XTS, we need to double the key size to 512bit

kdf = PBKDF2HMAC(
    # Commented below is what would have been used for CBC
    algorithm=hashes.SHA512_256(), # Faster than 256 on 64bit machines
    length=32,
    salt=os.urandom(32),
    iterations=1000000,
    backend=default_backend()
)

xtskdf = PBKDF2HMAC(
    # Commented below is what would have been used for CBC
    #algorithm=hashes.SHA512_256(), # Faster than 256 on 64bit machines
    #length=32,
    algorithm=hashes.SHA512(),
    length=64,
    salt=os.urandom(32),
    iterations=1000000,
    backend=default_backend()
)


In [ ]:
# "Cache" in the notebook
# Otherwise we get an exception for using the KDF twice
#key = kdf.derive(b"Password")
key = kdf.derive(os.urandom(16))
xtskey = xtskdf.derive(b"Password")

In [ ]:
# Construct an AES CBC Context

# Don't actually use that as the password!
algo = AES(key)

# We don't care about the IV
# We can use a dead block at the start
mode = CBC(os.urandom(16))

# Now we get our two functions
# We'll just use encryption for this but it's a good example
# NOTE: This can throw cryptography.exceptions.UnsupportedAlgorithm
cipher = Cipher(algo, mode, backend=default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
# XTS
def DoXTSSectorEncryption(key, secID):
    # We want to get the IV from disk sector if possible
    xtsalgo = AES(key)
    xtsmode = XTS(secID)
    



In [ ]:
def GetBlockDeviceSize(blk):
    sizeOfBlockDevice = 0
    try:
        sizeOfBlockDevice = os.lseek(blk, 0, os.SEEK_END)
    finally:
        os.lseek(blk, 0, os.SEEK_SET)
        
    return sizeOfBlockDevice

In [ ]:
def GetBlockDeviceSizeSafe(blk):
    rawBlkSize = GetBlockDeviceSize(blk)
    FullSafeSize = rawBlkSize // 16 # AES Block Size
    #sectors -= 1 # For Dead Block
    return FullSafeSize * 16
    

In [ ]:
phy = os.open("/dev/mmcblk0", os.O_RDWR)

diskSize = GetBlockDeviceSizeSafe(phy)

print(GetBlockDeviceSizeSafe(phy) / (1024 * 1024))

stride = 16
totalIndices = diskSize // stride

if not diskSize % stride == 0:
    print("FDE will not work here")
    

In [ ]:
# Prime the encryptor with a dead block
deadBlock = os.urandom(16)

shiftArray = [deadBlock, b""]

for i in range(0,totalIndices):
    shiftArray[1] = os.pread(phy, stride, i * stride)
    
    encData = encryptor.update(shiftArray[0])
    
    os.pwrite(phy, encData, i * stride)
    
    shiftArray[0] = shiftArray[1]
    
encryptor.finalize()
    
    
    


In [ ]:
decryptor.update(os.pread(phy, stride, 0))

array = b""
for i in range(1, 10):#totalIndices):
    array += decryptor.update(os.pread(phy, stride, i * stride))
    
print(array)